In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib as mptlib
import keras as ks
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import os

np.random.seed(42) # set the seed for reproducibility

2022-04-19 20:03:38.754009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 20:03:38.754036: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
root = "./"
df = pd.read_csv(root + "/oil_wells_data.csv")
df.head()

,timestamp,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,T-JUS-CKGL,QGL,class
0,2018-05-17 22:56:06.000000,27897490.0,19824230.0,125.6859,4059666.0,97.55283,NaN,NaN,NaN,0
1,2018-05-17 22:56:07.000000,27897450.0,19824230.0,125.6859,4059666.0,97.55283,NaN,NaN,NaN,0
2,2018-05-17 22:56:08.000000,27897360.0,19824230.0,125.6859,4059666.0,97.55283,NaN,NaN,NaN,0
3,2018-05-17 22:56:09.000000,27897430.0,19824230.0,125.6859,4059666.0,97.55282,NaN,NaN,NaN,0
4,2018-05-17 22:56:10.000000,27897500.0,19824230.0,125.6859,4059666.0,97.55282,NaN,NaN,NaN,0


In [3]:
#we drop the timestamps and the empty columns
clean_df = df.drop({'timestamp','P-JUS-CKGL','T-JUS-CKGL','QGL'}, axis=1)

#checking for any Nan value
array_sum = np.sum(clean_df)
array_has_nan = np.isnan(clean_df)
for i in array_has_nan:
    if i==True:
        print("True")


In [4]:
x = clean_df.drop({'class'}, axis = 1)
y = clean_df['class']

#we drop the first 25 lines because each timeserie will have the label of the last observation (25th) and our stride is one
y = y.drop({i for i in range(24)}, axis=0)
#resetting the index
y = y.reset_index()
# to drop the index columns
y = y['class']

# we apply a sliding window of size 2 and stride 1 on our dataset to generate multiple instances of timeseries.
timeseries_x = np.lib.stride_tricks.sliding_window_view(x, window_shape=(25,5))
timeseries_x = np.reshape(timeseries_x, (timeseries_x.shape[0], timeseries_x.shape[2]*5 ))

timeseries_x = pd.DataFrame(timeseries_x)

#we stick them back together:
timeseries = pd.concat([timeseries_x, y], axis=1, join='inner')
timeseries


,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,class
0,27897490.0,19824230.0,125.685900,4059666.0,97.55283,27897450.0,19824230.0,125.685900,4059666.0,97.55283,...,19824230.0,125.685900,4059666.0,97.55279,27897490.0,19824230.0,125.685900,4059666.0,97.55279,0
1,27897450.0,19824230.0,125.685900,4059666.0,97.55283,27897360.0,19824230.0,125.685900,4059666.0,97.55283,...,19824230.0,125.685900,4059666.0,97.55279,27897450.0,19824230.0,125.685900,4059666.0,97.55279,0
2,27897360.0,19824230.0,125.685900,4059666.0,97.55283,27897430.0,19824230.0,125.685900,4059666.0,97.55282,...,19824230.0,125.685900,4059666.0,97.55279,27897360.0,19824230.0,125.685900,4059666.0,97.55279,0
3,27897430.0,19824230.0,125.685900,4059666.0,97.55282,27897500.0,19824230.0,125.685900,4059666.0,97.55282,...,19824230.0,125.685900,4059666.0,97.55279,27897430.0,19824230.0,125.685900,4059666.0,97.55279,0
4,27897500.0,19824230.0,125.685900,4059666.0,97.55282,27897310.0,19824230.0,125.685900,4059666.0,97.55282,...,19824230.0,125.685900,4059666.0,97.55279,27897500.0,19824230.0,125.685900,4059666.0,97.55279,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,33700240.0,28074330.0,3.413718,4002678.0,28.57462,33700330.0,28074330.0,3.413700,4002681.0,28.54115,...,28074100.0,3.413910,4002698.0,28.03539,33700360.0,28074090.0,3.413923,4002707.0,28.00641,8
26971,33700330.0,28074330.0,3.413700,4002681.0,28.54115,33700270.0,28074290.0,3.413787,4002686.0,28.50973,...,28074090.0,3.413923,4002707.0,28.00641,33700220.0,28074160.0,3.413754,4002710.0,27.97934,8
26972,33700270.0,28074290.0,3.413787,4002686.0,28.50973,33700310.0,28074330.0,3.413671,4002693.0,28.48163,...,28074160.0,3.413754,4002710.0,27.97934,33700150.0,28074160.0,3.413741,4002710.0,27.95308,8
26973,33700310.0,28074330.0,3.413671,4002693.0,28.48163,33700270.0,28074310.0,3.413711,4002700.0,28.45841,...,28074160.0,3.413741,4002710.0,27.95308,33700210.0,28074220.0,3.413593,4002719.0,27.92784,8


In [5]:
from sklearn.model_selection import train_test_split

timeseries_train, timeseries_test = train_test_split(timeseries, test_size=0.25, random_state=42)
timeseries_train

,0,1,2,3,4,5,6,7,8,9,...,116,117,118,119,120,121,122,123,124,class
18967,33700300.0,28137840.0,3.419346,4002560.0,32.81291,33700300.0,28137830.0,3.419345,4002546.0,32.81478,...,28137630.0,3.419340,4002538.0,32.65928,33700290.0,28137620.0,3.419339,4002547.0,32.65839,8
3202,28581700.0,20762250.0,125.855600,4058672.0,97.43038,28584850.0,20766560.0,125.855800,4058668.0,97.43000,...,20869540.0,125.874100,4058590.0,97.42121,28664200.0,20874490.0,125.874800,4058586.0,97.42080,108
22142,33699970.0,28108010.0,3.419697,4002721.0,30.01394,33699970.0,28108010.0,3.419673,4002719.0,30.02527,...,28107790.0,3.419702,4002285.0,31.40709,33699970.0,28107800.0,3.419659,4002288.0,31.47476,8
26336,33700230.0,28078030.0,3.414448,4002738.0,28.10863,33700250.0,28078100.0,3.414289,4002737.0,28.10761,...,28077920.0,3.414353,4002701.0,28.38741,33700230.0,28077940.0,3.414306,4002701.0,28.38218,8
12181,33702020.0,28202130.0,3.395988,4002553.0,37.98204,33702020.0,28202120.0,3.396006,4002552.0,37.93012,...,28201960.0,3.396413,4002202.0,37.03981,33701990.0,28201950.0,3.396432,4002199.0,37.14220,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,33700110.0,28113660.0,3.419024,4002616.0,30.75280,33700110.0,28113650.0,3.419024,4002619.0,30.75782,...,28113460.0,3.419015,4002255.0,31.38537,33700110.0,28113450.0,3.419015,4002268.0,31.47936,8
5390,33694490.0,27443040.0,6.275269,4007067.0,80.55661,33694400.0,27432620.0,6.267845,4007065.0,80.54888,...,27455260.0,6.177568,4007023.0,80.37781,33689990.0,27459360.0,6.174535,4007021.0,80.36997,108
860,27897370.0,19824240.0,125.685800,4059666.0,97.55170,27897440.0,19824240.0,125.685800,4059666.0,97.55170,...,19824240.0,125.685800,4059666.0,97.55167,27897370.0,19824240.0,125.685800,4059666.0,97.55167,0
15795,33700760.0,28167050.0,3.420566,4002264.0,34.65635,33700760.0,28167040.0,3.420566,4002268.0,34.79261,...,28166820.0,3.420567,4002596.0,36.16253,33700750.0,28166810.0,3.420567,4002593.0,36.10564,8


In [48]:
#standardization
from sklearn.preprocessing import StandardScaler

def standardize_df(data):

    scaler = StandardScaler()
    #transforming the df to a numpy array
    df = data.values
    #back to pandas dataframe
    df_std = pd.DataFrame(scaler.fit_transform(df),
                            columns=data.columns, index=data.index)
    return df_std


# we drop the class labels
x_train = timeseries_train.drop({'class'}, axis=1)
x_test = timeseries_test.drop({'class'}, axis =1)

x_train = standardize_df(x_train)
x_test = standardize_df(x_test)

x_train = x_train.values
x_train = np.reshape(x_train, (x_train.shape[0], 25, 5 ))
x_test = x_test.values
x_test = np.reshape(x_test, (x_test.shape[0], 25, 5 ) )


In [49]:
y_train = timeseries_train['class']
y_train = pd.get_dummies(y_train)
y_test = timeseries_test['class']
y_test = pd.get_dummies(y_test)


In [50]:
# Implement the convolutional layer

conv = tf.keras.layers.Conv1D(filters=8, kernel_size=2, strides=1,
                              padding='same', activation='relu')

#maxpool = tf.keras.layers.MaxPooling1D(1)
avepool = tf.keras.layers.AveragePooling1D(1)

verbose, epochs, batch_size = 0, 10, 32
n_timesteps, n_features, n_outputs = 25, 5, 3
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=1, strides = 1, padding = 'same', activation='relu', input_shape=(n_timesteps,n_features)))

model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)


In [51]:
print(accuracy)


0.9552194476127625
